In [7]:
import numpy as np
import pandas as pd


data = pd.read_csv('C:/Users/lisia/Downloads/Laptop_price.csv')

storage_capacity_values = data['Storage_Capacity'].values
processor_speed_values = data['Processor_Speed'].values
screen_size_values = data['Screen_Size'].values

X = []

for i in range(len(storage_capacity_values)):
    X.append([storage_capacity_values[i], processor_speed_values[i], screen_size_values[i]])

X = np.array(X)
X = (X - X.mean()) / X.std()

X_train = X[:700]
X_test = X[-300:]

price_values = data['Price'].values

# print("Минимальное значение цены:", np.min(price_values))
# print("Максимальное значение цены:", np.max(price_values))

num_groups = 15
max_val = 35000
min_val = 7000

y = []

price_bins = np.linspace(min_val, max_val, num_groups + 1)


for price in price_values:
    group_index = np.argmax(price_bins > price) - 1
    group_vector = [0] * num_groups
    group_vector[group_index] = 1
    y.append(group_vector)

y = np.array(y)

y_train = y[:700]
y_test = y[-300:]

for x_val, y_val in zip(X_train, y_train):
    print(x_val, y_val)

[ 0.95459161 -0.60249994 -0.57996381] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[ 2.44988081 -0.60531115 -0.57957704] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[ 0.1701776  -0.60430369 -0.57791736] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.95459161 -0.60257362 -0.57660795] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[ 2.44988081 -0.60422258 -0.56830261] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[ 0.1701776  -0.60847174 -0.57763873] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.1701776  -0.6052257  -0.57834565] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.1701776  -0.60499262 -0.56647199] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.95459161 -0.60639127 -0.56819397] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[ 0.95459161 -0.60399886 -0.57252782] [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[ 2.44988081 -0.60248271 -0.57043223] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[ 0.95459161 -0.60255011 -0.57479539] [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[ 0.95459161 -0.60618667 -0.5716605 ] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[ 0.1701776 -0.6087728 -0.5712161] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.1701776  -0.6020960

In [8]:
learning_rate = 0.5
epochs = 1500

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def initialize_weights_and_biases(layer_sizes):
    np.random.seed(1)
    num_layers = len(layer_sizes)
    weights = []
    biases = []
    for i in range(1, num_layers):
        weights.append(np.random.uniform(size=(layer_sizes[i - 1], layer_sizes[i])))
        biases.append(np.random.uniform(size=(1, layer_sizes[i])))
    return weights, biases


def forward_propagation(X, weights, biases):
    num_layers = len(weights) + 1
    layer_outputs = [X]
    for i in range(num_layers - 1):
        layer_input = np.dot(layer_outputs[-1], weights[i]) + biases[i]
        layer_output = sigmoid(layer_input)
        layer_outputs.append(layer_output)
    return layer_outputs


def backward_propagation(X, y, layer_outputs, weights, biases, learning_rate):
    num_layers = len(weights)
    num_samples = X.shape[0]
    output_layer_output = layer_outputs[-1]
    output_error = y - output_layer_output
    output_delta = output_error * sigmoid_derivative(output_layer_output)
    for i in range(num_layers - 1, -1, -1):
        if i == num_layers - 1:
            layer_delta = output_delta
        else:
            layer_error = layer_delta.dot(weights[i + 1].T)
            layer_delta = layer_error * sigmoid_derivative(layer_outputs[i + 1])
        weights[i] += layer_outputs[i].T.dot(layer_delta) * learning_rate / num_samples
        biases[i] += np.sum(layer_delta, axis=0, keepdims=True) * learning_rate / num_samples
    return weights, biases


def train_network(X, y, layer_sizes, learning_rate, epochs):
    weights, biases = initialize_weights_and_biases(layer_sizes)
    for epoch in range(epochs):
        layer_outputs = forward_propagation(X, weights, biases)
        weights, biases = backward_propagation(X, y, layer_outputs, weights, biases, learning_rate)
    print("Training complete!")
    return weights, biases

layer_sizes = [X.shape[1], 8, y.shape[1]]

weights, biases = train_network(X_train, y_train, layer_sizes, learning_rate, epochs)

Training complete!


In [22]:
def predict(X, price_values, weights, biases):
    layer_outputs = forward_propagation(X, weights, biases)
    predictions = np.argmax(layer_outputs[-1], axis=1)
    length = (max_val - min_val) / num_groups
    start = predictions * length + min_val
    end = (predictions + 1) * length + min_val
    result = np.column_stack((price_values, start, end))
    return predictions, result

predictions, result = predict(X_test, price_values[-300:], weights, biases)
print("Prediction for new data:")

for res in result:
    print(res[1], " < ", res [0], " < ", res[2])


count = sum(1 for res in result if res[0] > res[1] and res[0] < res[2])
print("Accuracy:", count/300)

Prediction for new data:
8866.666666666666  <  10374.526469146394  <  10733.333333333334
31266.666666666668  <  31494.3586227087  <  33133.333333333336
16333.333333333334  <  17000.04279606425  <  18200.0
8866.666666666666  <  10881.506435734313  <  10733.333333333334
16333.333333333334  <  16839.889865827867  <  18200.0
16333.333333333334  <  16919.58957778899  <  18200.0
31266.666666666668  <  31727.413250211663  <  33133.333333333336
16333.333333333334  <  17817.927110290842  <  18200.0
31266.666666666668  <  32923.66424093799  <  33133.333333333336
8866.666666666666  <  9217.345503076524  <  10733.333333333334
16333.333333333334  <  17420.461668116226  <  18200.0
8866.666666666666  <  10668.03049422374  <  10733.333333333334
16333.333333333334  <  16908.293955737794  <  18200.0
8866.666666666666  <  10614.656373393547  <  10733.333333333334
16333.333333333334  <  18478.144403711227  <  18200.0
16333.333333333334  <  17159.322422840665  <  18200.0
16333.333333333334  <  17265.177331

In [23]:
predictions_array = np.zeros((len(predictions), 15), dtype=int)

for i, pred in enumerate(predictions):
    predictions_array[i, pred] = 1

for pred, y_val in zip(predictions_array, y_test):
    print(pred, y_val)

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] [0 0 0 0

In [32]:
TP = sum(np.logical_and(predictions_array[i] == 1, y_test[i] == 1).sum() for i in range(len(predictions_array)))
FP = sum(np.logical_and(predictions_array[i] == 0, y_test[i] == 1).sum() for i in range(len(predictions_array)))
FN = sum(np.logical_and(predictions_array[i] == 1, y_test[i] == 0).sum() for i in range(len(predictions_array)))

# print(TP)
# print(FP)
# print(FN)

precision = TP/(FP+TP)
recall = TP/(FN+TP)

print("Precision = ", precision)
print("Recall = ", recall)
print("F1-score = ", 2*recall*precision/(recall+precision))


267
33
33
Precision =  0.89
Recall =  0.89
F1-score =  0.89
